In [1]:
%matplotlib inline

from modsim import *

In [14]:
# Import relevant units

m = UNITS.meter
kg = UNITS.kilogram
s = UNITS.second
N = UNITS.newton
cm = UNITS.centimeter
gm = UNITS.gram
mm = UNITS.millimeter
degree = UNITS.degree

In [15]:
# Set initial conditions.  These conditions will be used in make_system to generate a system.

condition = Condition(y=0*m, 
                      g=9.8*m/s**2, 
                      diameter=4*cm, 
                      rho_air=1.2*kg/m**3, 
                      C_d=0.75, 
                      velocity=0*m/s, 
                      duration=100*s, 
                      engineSA=4.5234*cm**2,
                      m_engine=50.6360*gm,
                      m_rocket=50*gm,
                      rho_fuel=1.599*gm/cm**3,
                      burnRate=1*mm/s,
                      thetaCone=75*degree)

In [18]:
# Takes in an initial condition and generates a full system

def make_system(condition):
    unpack(condition)
    vy = velocity
    init = State(m_engine=m_engine,y=y,vy=vy)
    r = diameter/2
    h = r*math.tan(thetaCone)
    coneSA = 3.14159 * r * (r + math.sqrt(r*r + h*h))
    ts = linspace(0,duration,101)
    
    return System(init=init,
                  g=g,
                  rho_air=rho_air,
                  C_d=C_d,
                  engineSA=engineSA,
                  m_engine=m_engine,
                  m_rocket=m_rocket,
                  rho_fuel=rho_fuel,
                  burnRate=burnRate,
                  ts=ts,
                  coneSA=coneSA)

In [19]:
# Slope Function for OdeInt later

def slope_func(state,t,system):
    m_engine, y, vy = state
    unpack(system)
    m = m_rocket + m_engine
    m_dot = rho_fuel * engineSA * burnRate
    
    f_drag = rho_air * C_d * vy*vy * coneSA / 2
    a_drag = f_drag / m
    
    ay = (v_exhaust * m_dot) / m - g - a_drag
    return vy, ay

In [10]:
#test test